<a href="https://colab.research.google.com/github/anvisharma1/ObjectDetectionModel/blob/main/Object_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install mediapipe-model-maker

In [ ]:
from google.colab import files
import os
import json
import tensorflow as tf

from mediapipe_model_maker import object_detector

In [ ]:
import tensorflow as tf
print(tf.__version__)


2.15.0


In [ ]:
from google.colab import files
import os
import json
import tensorflow as tf

from mediapipe_model_maker import object_detector

In [ ]:
train_dataset_path = "blocks/train"
validation_dataset_path = "blocks/validation"

In [ ]:
with open(os.path.join(train_dataset_path, "labels.json"), "r") as f:
  labels_json = json.load(f)
for category_item in labels_json["categories"]:
  print(f"{category_item['id']}: {category_item['name']}")

FileNotFoundError: [Errno 2] No such file or directory: 'blocks/train/labels.json'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train_data = object_detector.Dataset.from_coco_folder(train_dataset_path, cache_dir="/tmp/od_data/train")
validation_data = object_detector.Dataset.from_coco_folder(validation_dataset_path, cache_dir="/tmp/od_data/validation")
print("train_data size: ", train_data.size)
print("validation_data size: ", validation_data.size)

FileNotFoundError: [Errno 2] No such file or directory: 'blocks/train/images'

In [ ]:
spec = object_detector.SupportedModels.MOBILENET_MULTI_AVG
hparams = object_detector.HParams(export_dir='exported_model')
options = object_detector.ObjectDetectorOptions(
    supported_model=spec,
    hparams=hparams
)

In [ ]:
model = object_detector.ObjectDetector.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


Model: "retina_net_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobile_net (MobileNet)      {'2': (None, 64, 64, 32   3704416   
                             ),                                  
                              '3': (None, 32, 32, 64             
                             ),                                  
                              '4': (None, 16, 16, 16             
                             0),                                 
                              '5': (None, 8, 8, 192)             
                             , '6': (None, 1, 1, 128             
                             0)}                                 
                                                                 
 fpn (FPN)                   {'5': (None, 8, 8, 128)   144928    
                             , '4': (None, 16, 16, 1             
                             28),                 

/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


19/19 [==============================] - 135s 5s/step - total_loss: 3.6153 - cls_loss: 3.2901 - box_loss: 0.0052 - model_loss: 3.5525 - val_total_loss: 1.4335 - val_cls_loss: 1.1863 - val_box_loss: 0.0037 - val_model_loss: 1.3706
Epoch 2/30
19/19 [==============================] - 86s 4s/step - total_loss: 1.3963 - cls_loss: 1.1525 - box_loss: 0.0036 - model_loss: 1.3334 - val_total_loss: 1.3683 - val_cls_loss: 1.1357 - val_box_loss: 0.0034 - val_model_loss: 1.3055
Epoch 3/30
19/19 [==============================] - 90s 5s/step - total_loss: 1.1710 - cls_loss: 1.0024 - box_loss: 0.0021 - model_loss: 1.1081 - val_total_loss: 1.3127 - val_cls_loss: 1.0528 - val_box_loss: 0.0039 - val_model_loss: 1.2498
Epoch 4/30
19/19 [==============================] - 95s 5s/step - total_loss: 0.9633 - cls_loss: 0.8265 - box_loss: 0.0015 - model_loss: 0.9004 - val_total_loss: 1.1903 - val_cls_loss: 0.9495 - val_box_loss: 0.0036 - val_model_loss: 1.1274
Epoch 5/30
19/19 [==============================] 

In [ ]:
loss, coco_metrics = model.evaluate(validation_data, batch_size=4)
print(f"Validation loss: {loss}")
print(f"Validation coco metrics: {coco_metrics}")

4/4 [==============================] - 5s 749ms/step - total_loss: 0.8741 - cls_loss: 0.6712 - box_loss: 0.0028 - model_loss: 0.8113
creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.07s).
Accumulating evaluation results...
DONE (t=0.05s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.231
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.402
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.220
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.231
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.371
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.516
 Average Recall     (AR) @[ 

In [ ]:
model.export_model()
!ls exported_model
files.download('exported_model/model.tflite')

Exporting a floating point model


/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


float_ckpt.data-00000-of-00001	float_ckpt.index  metadata.json  model.tflite


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install mediapipe_model_maker
from google.colab import files
import os
import json
import tensorflow as tf

from mediapipe_model_maker import object_detector
train_dataset_path = "blocks/train"
validation_dataset_path = "blocks/validation"

with open(os.path.join( train_dataset_path, "/labels.json"), "r") as f:
  labels_json = json.load(f)
#for category_item in labels_json["categories"]:
 # print(f"{category_item['id']}: {category_item['name']}")

train_data = object_detector.Dataset.from_coco_folder(train_dataset_path, cache_dir="/tmp/od_data/train")
validation_data = object_detector.Dataset.from_coco_folder(validation_dataset_path, cache_dir="/tmp/od_data/validation")
spec = object_detector.SupportedModels.MOBILENET_MULTI_AVG
hparams = object_detector.HParams(export_dir='exported_model')
options = object_detector.ObjectDetectorOptions(
    supported_model=spec,
    hparams=hparams
)

model = object_detector.ObjectDetector.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options)

model.export_model()
!ls exported_model
files.download('exported_model/model.tflite')

if category_item['id'] == 1:
  print("Brown Sunglasses")
if category_item['id'] == 2:
  print("Dark Blue Baseball Hat")
if category_item['id'] == 3:
  print(" White and pink Nike shoes")
if category_item['id'] == 4:
  print("Yellow Pikachu Lego")
if category_item['id'] == 5:
  print("AirPod Pros")


FileNotFoundError: [Errno 2] No such file or directory: '/labels.json'